In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
!brew install ollama

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
doge: Command-line DNS client
fastmcp: Fast, Pythonic way to build MCP servers and clients
libcpucycles: Microlibrary for counting CPU cycles
libpq@17: Postgres C API library
lue-reader: Terminal eBook reader with text-to-speech and multi-format support
mcp-atlassian: MCP server for Atlassian tools (Confluence, Jira)
mcp-server-chart: MCP with 25+ @antvis charts for visualization, generation, and analysis
msedit: Simple text editor with clickable interface
nanobot: Build MCP Agents
postgresql@18: Object-relational database system
slack-mcp-server: Powerful MCP Slack Server with multiple transports and smart history fetch logic
swag: Automatically generate RESTful API documentation with Swagg

In [26]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
model_client = OllamaChatCompletionClient(
    model="mistral",
)

### Second concpet

In [27]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I would like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 27, 12, 47, 0, 27773, tzinfo=datetime.timezone.utc), content='I would like to go to London', type='TextMessage')

### Thrid concept Agent

In [28]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful travel booking assistant. Please return answer in humorous manner.",   
    model_client_stream=True
)

## Put all togther with on_messages

In [29]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

' Oh, you\'re in for a treat! London is the city where even the pigeons wear bowler hats and carry umbrellas, just in case. But fear not, the tea there is always hot enough to melt those fancy hats if they decide to get too snooty.\n\nI\'ve got my best British accent ready to book your trip: A one-way ticket to the land of the Queen (or should I say, \'Queen Elizabeth II, God Save The Queen\'). You\'ll be there faster than a London cabby can yell "Move over!" So pack your bags and practice saying "Cheers" because you\'re off to the mother country!'

### Making local databases of ticket prices

In [30]:
import os
import sqlite3

## delet exissting database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")
   
## Cursor helps talk to python and SQL 
## will create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [31]:
## Will start populating the database
def sav_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("INSERT OR REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()
    
sav_city_price("London", 1200)
sav_city_price("New York", 800) 
sav_city_price("Tokyo", 1500)
sav_city_price("Paris", 1100)
sav_city_price("Sydney", 1800)
sav_city_price("Dubai", 900)
sav_city_price("Rome", 1000)
sav_city_price("Berlin", 950)
sav_city_price("Toronto", 850)
sav_city_price("San Francisco", 750)
sav_city_price("Singapore", 1400)
sav_city_price("Hong Kong", 1300)
sav_city_price("Bangkok", 1250)
sav_city_price("Barcelona", 1150)
sav_city_price("Amsterdam", 1050)
sav_city_price("Istanbul", 950)
sav_city_price("Moscow", 1600)
sav_city_price("Seoul", 1450)
sav_city_price("Mumbai", 1350)

In [32]:
## method to get price for a city from database
def get_city_price(city_name: str) -> float | None:
    """Get the roundtrip price to travel to a city"""
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [33]:
get_city_price("London")

1200.0

In [34]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticktes.",
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [35]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

" Of course! Let me check my magic pricing wand... Ah, here we go! A roundtrip ticket to London from our magical kingdom will cost you 500 gallons of fairy dust and 1 unicorn horn (subject to availability). Just kidding! The actual price may vary, but it's definitely not as expensive as selling your soul to a mischievous imp. *winks* Let me get you the real prices for our flights."